In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
from dmba import plotDecisionTree, classificationSummary, regressionSummary
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

import pandas as pd
from sklearn.model_selection import train_test_split


from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
breast_cancer_wisconsin_original = fetch_ucirepo(id=15) 
  
# data (as pandas dataframes) 
x = breast_cancer_wisconsin_original.data.features 
y = breast_cancer_wisconsin_original.data.targets 
  
# metadata 
print(breast_cancer_wisconsin_original.metadata) 
  
# variable information 
print(breast_cancer_wisconsin_original.variables) 


{'uci_id': 15, 'name': 'Breast Cancer Wisconsin (Original)', 'repository_url': 'https://archive.ics.uci.edu/dataset/15/breast+cancer+wisconsin+original', 'data_url': 'https://archive.ics.uci.edu/static/public/15/data.csv', 'abstract': 'Original Wisconsin Breast Cancer Database', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 699, 'num_features': 9, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['Class'], 'index_col': ['Sample_code_number'], 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1990, 'last_updated': 'Sun Mar 10 2024', 'dataset_doi': '10.24432/C5HP4Z', 'creators': ['WIlliam Wolberg'], 'intro_paper': None, 'additional_info': {'summary': "Samples arrive periodically as Dr. Wolberg reports his clinical cases. The database therefore reflects this chronological grouping of the data. This grouping information appears immediately below, having been removed fro

Fill NAs

In [2]:
from sklearn.impute import KNNImputer
data_c = x.copy()
imputer = KNNImputer(n_neighbors=25)
data_imputed = imputer.fit_transform(data_c)
data_imputed=pd.DataFrame(data_imputed,columns=data_c.columns)
data_imputed.isna().sum()

Clump_thickness                0
Uniformity_of_cell_size        0
Uniformity_of_cell_shape       0
Marginal_adhesion              0
Single_epithelial_cell_size    0
Bare_nuclei                    0
Bland_chromatin                0
Normal_nucleoli                0
Mitoses                        0
dtype: int64

In [3]:
x=data_imputed

Split data

In [4]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25, random_state=42)

Build Model for random forest

In [9]:
rf = RandomForestClassifier(n_estimators=10,random_state = 42)
rf.fit(x_train,y_train)
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_], axis=0)
df = pd.DataFrame({'feature': x_train.columns, 'importance': importances, 'std':std})
df = df.sort_values('importance')
print(df)

classificationSummary(y_test,rf.predict(x_test))

                       feature  importance       std
8                      Mitoses    0.001253  0.002085
4  Single_epithelial_cell_size    0.016851  0.011872
3            Marginal_adhesion    0.021160  0.018834
0              Clump_thickness    0.044982  0.023453
5                  Bare_nuclei    0.081021  0.046759
7              Normal_nucleoli    0.147170  0.248429
6              Bland_chromatin    0.160248  0.251080
2     Uniformity_of_cell_shape    0.258768  0.317336
1      Uniformity_of_cell_size    0.268548  0.312147
Confusion Matrix (Accuracy 0.9600)

       Prediction
Actual   0   1
     0 115   3
     1   4  53


/Users/connorbetancourt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
